# Combine the phenology analysis tiles

And mask regions such as urban, irrigated agriculture (we don't trust AusENDVI over these regions), and water bodies.

In [ ]:
%matplotlib inline

import os
import sys
import warnings

import numpy as np
import xarray as xr
import pandas as pd
from odc.geo.xr import assign_crs

sys.path.append('/g/data/os22/chad_tmp/AusEFlux/src/')
from _utils import round_coords


## Open masking datasets

In [ ]:
crops = xr.open_dataset('/g/data/os22/chad_tmp/Aus_phenology/data/croplands_5km.nc')['croplands']
crops = xr.where(crops==2, 0, 1) #irrigated crops
crops = round_coords(crops)

urban = xr.open_dataarray('/g/data/xc0/project/AusEFlux/data/urban_water_mask_5km.nc')
urban = ~urban
mask = (urban & crops)

## Join tiles together

and mask with urban etc.

In [ ]:
tiles_path = '/g/data/os22/chad_tmp/Aus_phenology/results/combined_tiles/'

def combine_tiles(path, v, mask):
    tiles = [path+i for i in os.listdir(path) if v in i]
    tiles = [xr.open_dataset(t) for t in tiles]
    p = xr.combine_by_coords(tiles)
    p = assign_crs(p,crs='EPSG:4326')
    
    for var in p.data_vars:
        try:
            del p[var].attrs['grid_mapping']
        except:
            continue

    # mask = (urban & crops)
    p = p.where(mask)
    return p

p_parcorr_ios = combine_tiles(tiles_path, 'IOS', mask)
# p_parcorr_ioc = combine_tiles(tiles_path, 'IOC', mask)
p_average = combine_tiles(tiles_path, 'mean', mask)
p_trends = combine_tiles(tiles_path, 'trends', mask)

# p_attribute_pls = combine_tiles(tiles_path, 'PLS', mask)
# p_attribute_ml = combine_tiles(tiles_path, 'ML', mask)
# p_attribute_pcmci = combine_tiles(tiles_path, 'PCMCI', mask)
# p_attribute_delta_slope = combine_tiles(tiles_path, 'delta_slope', mask)


## Interactive maps

In [ ]:
# p_average['SOS'].odc.explore(
#             tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
#            attr = 'Esri',
#            name = 'Esri Satellite',
#     cmap='twilight'
# )

In [ ]:
season_per_year = p_average['n_seasons']/p_average['n_years']
non_seasonal = xr.where((season_per_year <= 0.90),1,0)
extra_seasonal = xr.where((season_per_year >= 1.1),1,0)
seasonality_mask = (non_seasonal | extra_seasonal)
seasonality_mask.plot();

In [ ]:
import contextily as ctx
import matplotlib.pyplot as plt
v='POS'
sos_nan = np.isnan(p_trends[v+'_slope']).where(~np.isnan(p_average[v]))

fig,ax=plt.subplots(1,1, figsize=(6,6), layout='constrained')
xr.where((sos_nan==1) & (seasonality_mask==0), 1, np.nan).plot(ax=ax, add_colorbar=False, add_labels=False)
ctx.add_basemap(ax, source=ctx.providers.CartoDB.VoyagerNoLabels, crs='EPSG:4326', attribution='', attribution_size=1)
ax.axes.get_xaxis().set_ticks([])
ax.axes.get_yaxis().set_ticks([])
ax.set_title('Nans in '+v+' trends but not in seasonality mask');
print(np.sum(sos_nan) / np.sum(~np.isnan(p_average[v].where(seasonality_mask!=1))) * 100)

In [ ]:
# var = 'POS'
# cmap='RdBu_r'
# vmin, vmax= -2,2

# m = p_trends[var+'_slope'].odc.explore( #.where(p_trends[var+'_p_value']<=0.05) .where(seasonality_mask!=1)
#             tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
#             attr = 'Esri',
#             name = 'Esri Satellite',
#             cmap=cmap,
#             vmin=vmin,
#             vmax=vmax
# )
# m

## Export

In [ ]:
p_average.to_netcdf('/g/data/os22/chad_tmp/Aus_phenology/results/mean_phenology_perpixel_circular_final.nc')
p_trends.to_netcdf('/g/data/os22/chad_tmp/Aus_phenology/results/trends_phenology_perpixel_circular_final.nc')

p_parcorr_ios.to_netcdf('/g/data/os22/chad_tmp/Aus_phenology/results/ios_analysis_perpixel_circular_final.nc')
# p_parcorr_ioc.to_netcdf('/g/data/os22/chad_tmp/Aus_phenology/results/ioc_analysis_perpixel.nc')s

# p_attribute_pls.to_netcdf('/g/data/os22/chad_tmp/Aus_phenology/results/attribution_vPOS_PLS_perpixel.nc')
# p_attribute_ml.to_netcdf('/g/data/os22/chad_tmp/Aus_phenology/results/attribution_vPOS_ML_perpixel.nc')
# p_attribute_pcmci.to_netcdf('/g/data/os22/chad_tmp/Aus_phenology/results/attribution_vPOS_PCMCI_perpixel.nc')
# p_attribute_delta_slope.to_netcdf('/g/data/os22/chad_tmp/Aus_phenology/results/attribution_vPOS_delta_slope_perpixel.nc')